# Tests with Charles for Horqrux
I tested here also if the too long compilation was solved here in JAX
# Noise models in Global QNN variants.



In [3]:
import os
os.environ["JAX_CHECK_TRACER_LEAKS"] = "True"
# set env var JAX_CHECK_TRACER_LEAKS

In [4]:
!python -c "import jax; import qedft; print(jax.__version__); print(qedft.__version__)"

0.4.35
Traceback (most recent call last):
  File "<string>", line 1, in <module>
AttributeError: module 'qedft' has no attribute '__version__'


In [5]:
!which python

/Users/igorsokolov/PycharmProjects/qex/.venv/bin/python


In [6]:
import jax
from qedft.models.networks import LocalQNN, GlobalQNN, GlobalQNNwithMLP, GlobalQiCQNN, GlobalQiQNN
import jax.numpy as jnp

In [7]:
# Create different QNN models
models = {
    "DirectQNN": LocalQNN(config_dict={"qnn_type": "LocalQNN", "layer_type": "DirectQNN"}),
    "ProductQNN": LocalQNN(config_dict={"qnn_type": "LocalQNN", "layer_type": "ProductQNN"}),
}

# Setup test data
num_points = 9
density = jnp.linspace(0, 1, num_points)
print(f"Density shape: {density.shape}")
grids = jnp.ones(density.shape)
print(f"Grids shape: {grids.shape}")
rng_key = jax.random.PRNGKey(0)

# Test each model
results = {}
for name, model in models.items():
    # Build network
    init_fn, apply_fn = model.build_network(grids)

    # Initialize parameters and run inference
    _, params = init_fn(rng_key, input_shape=(-1, num_points, 1))
    output = apply_fn(params, density)

    # Store results
    results[name] = output

    # Print results
    print(f"\n=== {name} ===")
    print(f"Output shape: {output.shape}")
    print(f"Output values: {output}")

Density shape: (9,)
Grids shape: (9,)


2025-07-24 15:40:37.403 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building LocalQNN QNN with DirectQNN layer_type
2025-07-24 15:40:40.547 | INFO     | qedft.models.quantum.quantum_models:build_qnn:678 - ProductQNN can generate NaNs due to domain restriction of the arcsin function. Modify the map_fn (default: jnp.arcsin) to avoid this issue or normalize the input data.
2025-07-24 15:40:40.547 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building LocalQNN QNN with ProductQNN layer_type



=== DirectQNN ===
Output shape: (9,)
Output values: [1.95396874 1.93403894 1.86914238 1.76210312 1.61768188 1.44230961
 1.24372743 1.03055464 0.81181101]

=== ProductQNN ===
Output shape: (9,)
Output values: [1.97996765 1.9737837  1.92034164 1.81788923 1.66336047 1.451347
 1.17124902 0.79621778 0.05810413]


# Model 
We need the same Local QNN as in the convolutional in Qiskit I used.

For QNN in Qiskit I used this. 

This is a global model. It is the same in the paper. 
We can use the same as in the paper also for the noise study.
```python
def create_stax_model(n_qubits=2, n_layers=1, hidden_size=4, output_size=1):
    return stax.serial(
        QiskitQNNLayer(n_qubits, n_layers),  # quantum layer (trainable via param shift)
        stax.Dense(hidden_size),
        stax.Relu,
        stax.Dense(output_size),
    )
```

In [4]:
from qedft.models.quantum.convolutional_models import *

In [5]:
# #  Global QNN

# # Test configuration
# input_dimension = 513
# n_qubits = 6
# largest_kernel_dimension = n_qubits
# n_var_layers = 8
# n_out = 1
# max_number_conv_layers = 1

# # Get kernel dimensions and outputs per layer
# list_kernel_dimensions, list_outputs_per_conv_layer = compute_kernel_width_per_layer(
#     input_dimension=input_dimension,
#     largest_kernel_dimension=largest_kernel_dimension,
#     max_number_conv_layers=max_number_conv_layers,
# )
# print("Kernel dimensions:", list_kernel_dimensions)
# print("Outputs per layer:", list_outputs_per_conv_layer)

# # Construct model
# list_conv_layers = construct_convolutional_model(
#     n_qubits=n_qubits,
#     n_var_layers=n_var_layers,
#     n_out=n_out,
#     input_dimension=input_dimension,
#     largest_kernel_width=largest_kernel_dimension,
#     max_number_conv_layers=max_number_conv_layers,
#     use_qnn_layers=True,  #
#     use_bias_mlp=True,
#     use_amplitude_encoding=False,
# )

# # Create test input
# grids = jnp.ones(input_dimension)

# # Test each layer individually
# current_input = grids
# for i, (init_fn, apply_fn) in enumerate(list_conv_layers):
#     _, params = init_fn(jax.random.PRNGKey(i), (input_dimension,))
#     # print(params)
#     current_input = apply_fn(params, current_input)
#     print(f"Layer {i} output shape:", current_input.shape)

# # Test full model using stax
# network = stax.serial(*list_conv_layers)
# init_fn, apply_fn = network
# _, params = init_fn(jax.random.PRNGKey(0), (input_dimension,))

# # print(params)
# print(len(params[0]))
# print(len(params[1][0]))


# # Count parameters
# param_count = count_parameters(params)
# print("Number of parameters:", param_count)
# print("We want 316 parameters as in Global QNN")

# # Test forward pass
# output = apply_fn(params, grids)
# print("\nFull model output shape:", output.shape)
# print("Output:", output)

# # Test with JIT compilation
# jit_apply_fn = jax.jit(apply_fn)
# jit_output = jit_apply_fn(params, grids)
# print("\nJIT output matches:", jnp.allclose(output, jit_output))

In [6]:
# jit_output = jit_apply_fn(params, grids)
# jit_output

In [7]:
# Noise
from typing import Callable

import jax.numpy as jnp
import numpy as np
import pytest

from horqrux.api import expectation, run, sample
from horqrux.apply import apply_gates
from horqrux.noise import DigitalNoiseInstance, DigitalNoiseType
from horqrux.primitives.parametric import PHASE, RX, RY, RZ
from horqrux.primitives.primitive import NOT, H, I, S, T, X, Y, Z
from horqrux.utils.operator_utils import density_mat, product_state, random_state

MAX_QUBITS = 7
PARAMETRIC_GATES = (RX, RY, RZ, PHASE)
PRIMITIVE_GATES = (NOT, H, X, Y, Z, I, S, T)

NOISE_single_prob = (
    DigitalNoiseType.BITFLIP,
    DigitalNoiseType.PHASEFLIP,
    DigitalNoiseType.DEPOLARIZING,
    DigitalNoiseType.AMPLITUDE_DAMPING,
    DigitalNoiseType.PHASE_DAMPING,
)
ALL_NOISES = list(DigitalNoiseType)


def noise_instance(noise_type: DigitalNoiseType) -> DigitalNoiseInstance:
    if noise_type in NOISE_single_prob:
        errors = 0.1
    elif noise_type == DigitalNoiseType.PAULI_CHANNEL:
        errors = (0.4, 0.5, 0.1)
    else:
        errors = (0.2, 0.8)

    return DigitalNoiseInstance(noise_type, error_probability=errors)

In [8]:
# noise = (DigitalNoiseInstance(DigitalNoiseType.DEPOLARIZING, 0.1),)
# ops = [X(0, noise=noise), X(1)]
# state = product_state("00")
# # state_output = run(ops, state)
# # test sampling
# dm_state = density_mat(state)
# sampling_output = sample(
#     dm_state,
#     ops,
# )

In [9]:
import jax
from horqrux import QuantumCircuit, X, apply_gates, expectation, random_state, zero_state
from qedft.models.quantum.measurement import (
    qubit_magnetization,
    total_magnetization,
    total_magnetization_ops,
    total_magnetization_via_inner_product,
)
# Create noise instances
low_noise = (DigitalNoiseInstance(DigitalNoiseType.BITFLIP, 0.0), )
high_noise = (DigitalNoiseInstance(DigitalNoiseType.BITFLIP, 0.01),)


# Create circuits with different noise levels
gates_no_noise = [RX("x", 0), RX("y", 1)]
gates_low_noise = [RX("x", 0, noise=low_noise), RX("y", 1, noise=low_noise)]
gates_high_noise = [RX("x", 0, noise=high_noise), RX("y", 1, noise=high_noise)]

circuit_no_noise = QuantumCircuit(4, gates_no_noise)
circuit_low_noise = QuantumCircuit(4, gates_low_noise)
circuit_high_noise = QuantumCircuit(4, gates_high_noise)

# Create measurement operators
z_ops = total_magnetization_ops(4)

# Parameters for parametric gates
values = {"x": jnp.array([0.0]), "y": jnp.array([0.0])}

# Test with zero state
state = zero_state(4)
key = jax.random.PRNGKey(0)

mag_no_noise = expectation(state, circuit_no_noise, z_ops, values, key=key)
mag_low_noise = expectation(state, circuit_low_noise, z_ops, values, key=key)
mag_high_noise = expectation(state, circuit_high_noise, z_ops, values, key=key)

# Zero noise should match no noise
np.testing.assert_allclose(jnp.sum(mag_no_noise), jnp.sum(mag_low_noise), rtol=1e-5)

In [10]:
print(mag_no_noise)
print(mag_low_noise)
print(mag_high_noise)

[1. 1. 1. 1.]
[1. 1. 1. 1.]
[0.98 0.98 1.   1.  ]


In [11]:
n_qubits = 2

bitflip_rate = 0.2 # 0.2
amplitude_damping_rate = 0.08 # 0.08
phase_damping_rate = 0.05 # 0.05

noise_scaling = 1e-6

bitflip_rate *= noise_scaling
amplitude_damping_rate *= noise_scaling
phase_damping_rate *= noise_scaling

# Print the noise rates
print(f"Bitflip rate: {bitflip_rate}")
print(f"Amplitude damping rate: {amplitude_damping_rate}")
print(f"Phase damping rate: {phase_damping_rate}")

real_noise = (DigitalNoiseInstance(DigitalNoiseType.BITFLIP, bitflip_rate), DigitalNoiseInstance(DigitalNoiseType.AMPLITUDE_DAMPING, amplitude_damping_rate), DigitalNoiseInstance(DigitalNoiseType.PHASE_DAMPING, phase_damping_rate),)

# real_noise = (DigitalNoiseInstance(DigitalNoiseType.BITFLIP, 0.5), )


# Create measurement operators
z_ops = total_magnetization_ops(n_qubits)
print(z_ops)

# Parameters for parametric gates
values = {"x": jnp.array([0.0]), "y": jnp.array([0.0])}

# Test with zero state
state = zero_state(n_qubits)

key = jax.random.PRNGKey(0)

gates_real_noise = [RX("x", 0, noise=real_noise), RX("y", 1, noise=real_noise)]
circuit_real_noise = QuantumCircuit(n_qubits, gates_real_noise)
mag_real_noise = expectation(state, circuit_real_noise, z_ops, values, key=key)

print(mag_real_noise)

Bitflip rate: 2e-07
Amplitude damping rate: 8e-08
Phase damping rate: 5e-08
[Observable(operations=[Z(target=((0,),), control=((None,),))]), Observable(operations=[Z(target=((1,),), control=((None,),))])]
[0.9999996 0.9999996]


# Working example of the Global QNN with noise, DO NOT USE SHOTS

In [8]:
import os
from pathlib import Path
import jax
import jax.numpy as jnp
import qedft
from qedft.models.networks import GlobalQiCQNN, GlobalQiQNN, GlobalQNN
from qedft.config.config import Config
from qedft.train.od.trainer import KSDFTTrainer
from horqrux.utils.operator_utils import DiffMode
from loguru import logger

# Make sure we use double precision
jax.config.update("jax_enable_x64", True)

# Get project path
project_path = Path(os.path.dirname(os.path.dirname(qedft.__file__)))

# Load base configuration
config = Config(config_path=project_path / 'qedft' / 'config' / 'train_config.yaml').config

# Let's change to settings to have a global functional
config['network_type'] = 'conv_dqc'
config['use_amplitude_encoding'] = True

# Configure GlobalQNN
qicnn_config = {
    "network_type": "conv_dqc",
    "wrap_self_interaction": False,
    "wrap_with_negative_transform": False,
    "use_amplitude_encoding": True,
    "n_qubits": 6,  # Number of qubits in the quantum layer
    "n_var_layers": 8,  # Number of quantum layers
    "n_layers": 8,
    "largest_kernel_width": 4,
    "max_number_conv_layers": 1,  # Limit to 1 convolutional layer
    "list_qubits_per_layer": [],
    "force_qubits_per_layer_is_kernel_width": False,
    "normalization": 1.0,
    "last_layer_type": "dense",  # Use a linear layer at the end
    "use_bias_mlp": False,
    "last_layer_features": [1],
    "diff_mode": DiffMode.AD,
    "n_shots": 0,
    "key": jax.random.PRNGKey(0)
}

# Initialize the QiCNN network
network = GlobalQNN(config_dict=qicnn_config)

num_points = 513
density = jnp.linspace(0, 1, num_points)
init_fn, apply_fn = network.build_network(grids=density)

# Initialize parameters and run inference
rng_key = qicnn_config['key']
_, params = init_fn(rng_key, input_shape=(-1, num_points, 1))
output = apply_fn(params, density)

# Print the output
print(output)

# Initialize trainer
trainer = KSDFTTrainer(
    config_dict=config,
    network=network,
    data_path=project_path / 'data' / 'od'
)

# Train model
params, loss, info = trainer.train(
    checkpoint_save_dir=project_path / 'tests' / 'ckpts'
)


2025-07-24 15:41:01.827 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:180 - Using GlobalQNNLayer layer
2025-07-24 15:41:01.845 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:210 - GlobalQNNLayer Layer 0: n_qubits_layer 6
2025-07-24 15:41:02.020 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 15:41:02.020 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)


list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 15:41:52.963 | INFO     | qedft.train.od.trainer:__init__:58 - Initialized trainer with config: {'name': 'test', 'experiment_name': 'test', 'network_type': 'conv_dqc', 'molecule_name': 'h2', 'molecule_names': ['h2'], 'dataset1': [128, 384], 'rng': 0, 'save_plot_loss': False, 'save_every_n': 20, 'activation': 'tanh', 'n_neurons': 513, 'n_layers': 2, 'n_qubits': 9, 'n_reupload_layers': 1, 'use_rzz_parametrized_entanglers': False, 'chebychev_reuploading': False, 'add_reversed_rzz': False, 'entangling_block_type': 'alternate_linear', 'single_qubit_rotations': ['rz', 'rx', 'rz'], 'use_same_parameters': False, 'add_negative_transform': False, 'wrap_with_self_interaction_layer': False, 'wrap_with_global_functional': False, 'use_correlators_in_output': False, 'output_operators': ['Z'], 'use_bias_in_output': False, 'max_train_steps': 10000, 'factr': 1.0, 'pgtol': 1e-14, 'm': 20, 'maxfun': 20, 'maxiter': 2, 'num_iterations': 15, 'ks_iter_to_ignore': 10, 'discount_factor': 0.9, 'alpha'

[6.12796035]
list_kernel_dimensions , list_outputs_per_conv_layer:  [3] [171]


2025-07-24 15:41:53.236 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building GlobalQNN QNN with DirectQNN layer_type
2025-07-24 15:41:53.236 | INFO     | qedft.models.quantum.convolutional_models:construct_convolutional_model:268 - Adding a single dense layer at the end (outputs last layer 171)
2025-07-24 15:41:53.237 | INFO     | qedft.models.wrappers:wrap_network:205 - Global model, ensuring output is scalar (wrap_self_interaction causes global models to output (num_grids,) instead of (1,))
2025-07-24 15:41:53.237 | INFO     | qedft.train.od.trainer:train:108 - Initializing fresh parameters
2025-07-24 15:41:53.238 | INFO     | qedft.train.od.trainer:train:114 - Jitting neural_xc_energy_density_fn
2025-07-24 15:41:53.283 | INFO     | qedft.train.od.trainer:train:140 - Jitting loss_fn
2025-07-24 15:41:53.284 | INFO     | qedft.train.od.trainer:train:151 - Checkpoint save directory: /Users/igorsokolov/PycharmProjects/qex/tests/ckpts
2025-07-24 15:41:53.284 | INFO   

# Data when we managed to solve the problem of the infinite compilation


# Example where we have a Bug with GPSR and shots > 0

In [18]:
# import os
# from pathlib import Path
# import jax
# import jax.numpy as jnp
# import qedft
# from qedft.models.networks import GlobalQiCQNN, GlobalQiQNN, GlobalQNN
# from qedft.config.config import Config
# from qedft.train.od.trainer import KSDFTTrainer
# from horqrux.utils.operator_utils import DiffMode
# from loguru import logger

# # Make sure we use double precision
# jax.config.update("jax_enable_x64", True)

# # Get project path
# project_path = Path(os.path.dirname(os.path.dirname(qedft.__file__)))

# # Load base configuration
# config = Config(config_path=project_path / 'qedft' / 'config' / 'train_config.yaml').config

# # Let's change to settings to have a global functional
# config['network_type'] = 'dqc'
# config['use_amplitude_encoding'] = False

# # Configure GlobalQNN
# qicnn_config = {
#     "network_type": "conv_dqc",
#     "wrap_self_interaction": False,
#     "wrap_with_negative_transform": False,
#     "use_amplitude_encoding": False,
#     "n_qubits": 2,  # Number of qubits in the quantum layer
#     "n_var_layers": 2,  # Number of quantum layers
#     "n_layers": 2,
#     "largest_kernel_width": 4,
#     "max_number_conv_layers": 1,  # Limit to 1 convolutional layer
#     "list_qubits_per_layer": [],
#     "force_qubits_per_layer_is_kernel_width": False,
#     "normalization": 1.0,
#     "last_layer_type": "dense",  # Use a linear layer at the end
#     "use_bias_mlp": False,
#     "last_layer_features": [1],
#     "diff_mode": DiffMode.GPSR,
#     "n_shots": 100,
#     "key": jax.random.PRNGKey(0),
#     "qnn_type": "LocalQNN",
#     "layer_type": "DirectQNN"
# }
# # Initialize the QiCNN network
# # network = GlobalQNN(config_dict=qicnn_config)
# network = LocalQNN(config_dict=qicnn_config)


# # Noise
# bitflip_rate = 0.2 # 0.2
# amplitude_damping_rate = 0.08 # 0.08
# phase_damping_rate = 0.05 # 0.05

# noise_scaling = 1. # 1e-6

# bitflip_rate *= noise_scaling
# amplitude_damping_rate *= noise_scaling
# phase_damping_rate *= noise_scaling

# # Print the noise rates
# print(f"Bitflip rate: {bitflip_rate}")
# print(f"Amplitude damping rate: {amplitude_damping_rate}")
# print(f"Phase damping rate: {phase_damping_rate}")

# noise = (DigitalNoiseInstance(DigitalNoiseType.BITFLIP, bitflip_rate), DigitalNoiseInstance(DigitalNoiseType.AMPLITUDE_DAMPING, amplitude_damping_rate), DigitalNoiseInstance(DigitalNoiseType.PHASE_DAMPING, phase_damping_rate),)

# # noise = (DigitalNoiseInstance(DigitalNoiseType.BITFLIP, 0.0), )

# num_points = 513
# density = jnp.linspace(0, 1, num_points)
# init_fn, apply_fn = network.build_network(grids=density, noise=noise)

# print("Built the network")

# # Initialize parameters and run inference
# rng_key = qicnn_config['key']
# _, params = init_fn(rng_key, input_shape=(-1, num_points, 1))
# output = apply_fn(params, density)
# # Print the output
# print(output)

# # JIT the network
# jit_apply_fn = jax.jit(apply_fn)

# rng_key = qicnn_config['key']
# _, params = init_fn(rng_key, input_shape=(-1, num_points, 1))
# output = jit_apply_fn(params, density)
# print(output)


# # Initialize trainer
# trainer = KSDFTTrainer(
#     config_dict=config,
#     network=network,
#     data_path=project_path / 'data' / 'od'
# )

# # Train model
# params, loss, info = trainer.train(
#     checkpoint_save_dir=project_path / 'tests' / 'ckpts'
# )


2025-06-24 15:04:19.523 | INFO     | qedft.models.quantum.quantum_models:build_qnn:683 - Building LocalQNN QNN with DirectQNN layer_type


Bitflip rate: 0.2
Amplitude damping rate: 0.08
Phase damping rate: 0.05
Built the network
[0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.72 0.72 0.72 0.72
 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72
 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72
 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72
 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.72 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68
 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.68 0.64 0.64 0.64 0.64 0.64 0.64
 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.

Exception: Leaked trace MainTrace(2,BatchTrace). Leaked tracer(s):

Traced<ShapedArray(float64[])>with<BatchTrace(level=2/0)> with
  val = Traced<ShapedArray(float64[513])>with<DynamicJaxprTrace(level=1/0)>
  batch_dim = 0
This BatchTracer with object id 6048091008 was created on line:
  /Users/igorsokolov/PycharmProjects/qex/qedft/models/quantum/feature_maps.py:221 (<listcomp>)
<BatchTracer 6048091008> is referred to by <Parametric 6143206800>.param
<Parametric 6143206800> is referred to by <list 6142689216>[1]
<list 6142689216> is referred to by <Unhashable 6143204208>
<Unhashable 6143204208> is referred to by <tuple 13930302816>[1]
<tuple 13930302816> is referred to by <tuple 5685653664>[0]
<tuple 5685653664> is referred to by <tuple 6143323136>[1]
<tuple 6143323136> is referred to by <tuple 6141947776>[2]
<tuple 6141947776> is referred to by <WrappedFun 13930313216>
<WrappedFun 13930313216> is referred to by <function 6139539616> (jvp_jaxpr_thunk) closed-over variable jvp
<function

# Example where we will use just the Global QNN and noise there
- We will add the shot noise as a Gaussian on the output of the QNN